In [36]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

In [37]:
# train = pd.read_csv('input/train.csv')
# macro = pd.read_csv('input/macro.csv')
# test = pd.read_csv('input/test.csv')
# train = pd.merge(train, macro,  how='left', left_on=['timestamp'], right_on = ['timestamp'])

train = pd.read_csv('output/train.csv')

train_X = train.drop(["id", "timestamp", "price_doc"], axis=1)
train_y = np.log1p(train.price_doc.values)
train

,Unnamed: 0,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,lon,lat
0,8056,8059,512,20.0,16.0,2.0,5.0,2.0,1907.0,1.0,...,26,133,207,1,89,161,10,2750000.0,37.504051,55.525648
1,8135,8138,516,53.0,30.0,10.0,16.0,1.0,1980.0,2.0,...,0,11,21,1,0,46,5,9000000.0,37.786234,55.792259
2,8153,8156,517,77.0,41.0,2.0,17.0,6.0,2014.0,3.0,...,0,1,7,1,0,12,1,7011550.0,37.425744,55.721025
3,8154,8157,517,45.0,27.0,6.0,9.0,1.0,1970.0,2.0,...,0,3,8,1,0,19,3,7100000.0,37.789424,55.757179
4,8175,8178,518,38.0,20.0,15.0,16.0,1.0,1982.0,1.0,...,1,11,25,1,7,95,4,6450000.0,37.926360,55.703458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15550,30464,30467,1160,56.0,51.0,19.0,19.0,1.0,2017.0,2.0,...,0,14,29,1,4,90,6,10262010.0,37.520855,55.669737
15551,30466,30469,1160,44.0,27.0,7.0,9.0,1.0,1975.0,2.0,...,0,15,26,1,2,84,6,7400000.0,37.433281,55.862778
15552,30467,30470,1160,86.0,59.0,3.0,9.0,2.0,1935.0,4.0,...,24,98,182,1,82,171,15,25000000.0,37.636451,55.848884
15553,30469,30472,1160,64.0,32.0,5.0,15.0,1.0,2003.0,2.0,...,1,6,31,1,4,65,7,13500000.0,37.606609,55.868076


In [38]:
# A parameter grid for XGBoost
# can add more range
params = {
    'eta': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10],
    'objective': ['reg:squarederror'],
}

In [24]:
from sklearn.metrics import SCORERS
print(sorted(SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'brier_score_loss', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score']


In [39]:
xgb = XGBRegressor()

folds = 5
param_comb = 5 # this means randomly pick 5 combinations in params. can be larger

kf = KFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, cv=kf.split(train_X, train_y), verbose=3, random_state=1001)

# Here we go
random_search.fit(train_X, train_y)



Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0, score=0.317, total=   5.2s
[CV] subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s


[CV]  subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0, score=0.305, total=   5.1s
[CV] subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.3s remaining:    0.0s


[CV]  subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0, score=0.355, total=   5.2s
[CV] subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0 
[CV]  subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0, score=0.319, total=   5.3s
[CV] subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0 
[CV]  subsample=0.7, objective=reg:squarederror, min_child_weight=1, max_depth=5, eta=0.2, colsample_bytree=1.0, score=0.298, total=   5.7s
[CV] subsample=0.5, objective=reg:squarederror, min_child_weight=5, max_depth=7, eta=0.1, colsample_bytree=0.5 
[CV]  subsample=0.5, objective=reg:squarederror, min_child_weight=5, max_depth=7, eta=0.1, colsample_bytree=0.5, score=0.334, total=   3.9s
[CV] subsample=0.5, objective=reg:squarederror, min_child_weight=5, max_depth=7, eta=0.1, colsample_bytr

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.4min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7ff7cf408c50>,
                   error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=Non...
                                          validate_parameters=None,
                                          verbosity=None),
                   iid='warn', n_iter=5, n_jobs=None,
                   param_distributions={'colsample_bytree': [0.5, 0.7, 1.0],
                                       

In [40]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, eta=0.2, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.200000003, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='neg_mean_squared_error', cv=kf.split(train_X, train_y), verbose=3 )
grid.fit(train_X, train_y)
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results-01.csv', index=False)